In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

equivalencias = {
    "Tratamento De Disfunção Erétil": "Disfunção Erétil",
    "Redutor De Colesterol Antihipertensivo": "Colesterol E Triglicerídeos",
    "Analgésico Antitérmico": "Analgésicos",
    "Antidiabéticos": "Diabetes",
    "Anti-inflamatório": "Anti-Inflamatórios",
    "Corticóide Anti-inflamatório": "Anti-Inflamatórios",
    "Calmante Tranquilizante": "Calmantes",
    "Anti Histaminico E Antialergico Topico": "Alergias E Infecções",
    "Anti-Infeccioso": "Alergias E Infecções",
    "Anti-Infeccioso Ocular": "Alergias E Infecções",
    "Anti-Hipertensivos": "Hipertensão",
    "Anti-Infeccioso Topico Cicatrizante": "Alergias E Infecções",
    "Laxantes e Purgantes": "Digestivo",
    "Regulador Intestinal": "Emagrecedores",
    "Antigripal": "Gripe E Resfriado",
    "Anticoncepcional": "Anticoncepcionais",
    "Antigases Antiespasmódico": "Digestivo",
    "Antiviral": "Gripe E Resfriado",
    "Descongestionante Nasal": "Gripe E Resfriado",
    "Rehidratante": "Similar",
    "Antimicotico": "Similar",
    "Cicatrizante": "Similar",
    "Dermatológico": "Similar",
    "Orexígeno": "Similar",
    "Antiulceroso": "Dor E Febre",
    "Corticóide Tópico": "Anti-Inflamatórios",
    "Tratamento De Enxaqueca": "Dor E Febre",
    "VITAMINAS": "Vitaminas",
    # O restante dos medicamentos serão agrupados como "Mais Medicamentos"
    "Anestésico Tópico": "Mais Medicamentos",
    "Anti-Hemorragico": "Mais Medicamentos",
    "Antiacneico": "Mais Medicamentos",
    "Antiarritmico": "Mais Medicamentos",
    "Antiartritico": "Mais Medicamentos",
    "Antiasmático": "Mais Medicamentos",
    "Antibiotico": "Mais Medicamentos",
    "Anticoagulante": "Mais Medicamentos",
    "Antidepressivo": "Mais Medicamentos",
    "Antidiarreico": "Mais Medicamentos",
    "Antiemetico": "Mais Medicamentos",
    "Antiespamódico Urinário": "Mais Medicamentos",
    "Antiglaucomatoso": "Mais Medicamentos",
    "Antigotoso": "Mais Medicamentos",
    "Antihemorroidal": "Mais Medicamentos",
    "Antineoplasico": "Mais Medicamentos",
    "Antineuritico": "Mais Medicamentos",
    "Antiparasitario": "Mais Medicamentos",
    "Antiparkinsoniano": "Mais Medicamentos",
    "Antisséptico": "Mais Medicamentos",
    "Antisséptico Vias Urinárias": "Mais Medicamentos",
    "Antitabagismo": "Mais Medicamentos",
    "Antitrombotico": "Mais Medicamentos",
    "Antitussigeno": "Mais Medicamentos",
    "Antivaricoso": "Mais Medicamentos",
    "Antivertiginoso": "Mais Medicamentos",
    "Broncodilatador": "Mais Medicamentos",
    "Calcificantes": "Mais Medicamentos",
    "Ceratolitico": "Mais Medicamentos",
    "Diurético": "Mais Medicamentos",
    "Escabicida": "Mais Medicamentos",
    "Hepatoprotetor": "Mais Medicamentos",
    "Hipnóticos Sedativos": "Mais Medicamentos",
    "Imunizante": "Mais Medicamentos",
    "Imuno Modulador": "Mais Medicamentos",
    "Manutenção Óssea": "Mais Medicamentos",
    "Mucolítico": "Mais Medicamentos",
    "Psicotropico": "Mais Medicamentos",
    "Reumatismo": "Mais Medicamentos",
    "Tratamento De Distúrbios Hormonais": "Mais Medicamentos",
    "Vasodilatador Periferico": "Mais Medicamentos"
}

def trocar_valor(valor, equivalencias):
    if valor in equivalencias:
        return equivalencias[valor]
    else:
        return valor

def fetch_medicamento(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        return None

def process_medicamento(medicamento, lista_marcas):
    try:
        nome = medicamento.find("h1", class_="name").text.strip()
    except:
        nome = 'sem'
    try: 
        preco = float(medicamento.find("p", class_="unit-price").text.replace('R$ ', '').replace(',', '.'))
    except: 
        preco = 'sem'
    try:
        ean = medicamento.find(itemprop="gtin13").get('content')
    except:
        ean = 'sem'
    try: 
        desconto = float(medicamento.find("span", class_="discount").find('b').text.replace('%', ''))
    except: 
        desconto = 'sem'
    try:
        preco_desconto = round(preco * (1-desconto/100), 2)
    except:
        preco_desconto = 'sem'
    try:
        categoria = medicamento.find("li", class_="pr-0 mr-0").find('b').text
        categoria = trocar_valor(categoria,equivalencias)
    except:
        categoria = 'sem'
    # Procurar pela marca no HTML
    marca_html = medicamento.find("a", class_="text-primary font-weight-bold")
    if marca_html:
        marca = marca_html.text.strip()
    else:
        # Procurar pela marca no nome do medicamento
        for marca in lista_marcas:
            if marca in nome:
                break
        else:
            marca = None
    return nome, preco, preco_desconto, desconto, ean, marca, categoria



def main():
    URLS=[]
    urls_primaria = []
    for i in range(1, 229):
        # URL da página de medicamentos
        urls_primaria.append("https://www.farmaponte.com.br/medicamentos/?p="+str(i))

    with ThreadPoolExecutor(max_workers=10) as executor:
        responses_primaria = list(executor.map(fetch_medicamento, urls_primaria))
        
        for response in responses_primaria:
            # Criar objeto BeautifulSoup
            soup = BeautifulSoup(response, "html.parser")
            # Encontrar os containers dos medicamentos
            medicamentos = soup.find_all("div", class_="item-product")
            # Extrair dados para cada medicamento
            for medicamento in medicamentos:
                nome = medicamento.find("h2", class_="title").text.strip()
                try:
                    preco = medicamento.find("p", class_="pix-price").text.strip()
                except: 
                    preco='sem'
                URL = "https://www.farmaponte.com.br" + medicamento.find("a", class_='item-image').get('href')
                # Imprimir dados
                URLS.append(URL)
                
    print(len(URLS))
    print(URLS[0])

    NOME = []
    PRECO=[]
    PRECO_DESCONTO = []
    DESCONTO = []
    EAN=[]
    MARCA=[]
    CATEGORIA=[]
    lista_marcas = ["1Farma", "ADV Farma", "Abbott do Brasil", "Accord Farma", "Aché",
    "Addera", "Allergan", "Althaia", "Apsen", "Arese Pharma", "Aspen Pharma",
    "Astellas Farma", "Astrazeneca", "Baldacci", "Bausch + Lomb", "Bayer",
    "Besins", "Biolab", "Biolab Genéricos", "Biosintética - Aché", "Blanver S.A.",
    "Blau", "Boehringer Ingelheim", "Boiron", "Brasterápica", "Bravir",
    "Catarinense Pharma", "Cellera Farma", "Centrum", "Chiesi", "Cifarma",
    "Cimed", "Cosmed", "Daiichi-Sankyo", "Diffucap Chemobras", "EMS",
    "EMS Sigma Pharma", "Eli Lilly", "Eurofarma", "Exeltis", "Farmax",
    "Farmoquímica", "FiberMais", "GSK", "Galderma", "Geolab", "Germed Pharma",
    "Glenmark", "Granado Pharmácias", "Gross", "Grünenthal", "Gummy", "Hebron",
    "Herbarium", "Janssen-Cilag", "Kley Hertz Farmacêutica", "Laboratório Cristália",
    "Legrand", "Leo Pharma", "Libbs", "Marjan Farma", "Medley", "Medquímica",
    "Megalabs", "Merck", "Merck Sharp & Dohme", "Momenta Farma", "Multilab", "Mylan",
    "Myralis", "Natulab", "Neo Química", "Nova Química", "Novartis", "Novo Nordisk",
    "Organon", "P&G", "PF Consumer Healthcare", "Pfizer", "Pharlab", "Polinize Florais",
    "Prati-Donaduzzi", "Ranbaxy", "Reckitt Benckiser", "Roche", "Sandoz", "Sanofi",
    "Servier", "Supera", "Takeda", "Teuto", "Theraskin", "Top Beauty", "Torrent",
    "União Química", "Vick", "Vitamedic", "Wyeth/Pfizer", "Zambon", "Zodiac"]  # Adicione suas marcas aqui
    with ThreadPoolExecutor(max_workers=12) as executor:
        responses = list(executor.map(fetch_medicamento, URLS))
        
        for response in responses:
            soup = BeautifulSoup(response, "html.parser")
            medicamentos = soup.find_all("div", class_="content")
            for medicamento in medicamentos:
                nome, preco, preco_desconto, desconto, ean, marca, categoria = process_medicamento(medicamento, lista_marcas)
                NOME.append(nome)
                PRECO.append(preco)
                PRECO_DESCONTO.append(preco_desconto)
                DESCONTO.append(desconto)
                EAN.append(ean)
                MARCA.append(marca)
                CATEGORIA.append(categoria)

    df = pd.DataFrame()
    df['Nome'] = NOME
    df['Preço sem desconto'] = PRECO
    df['Preço com desconto'] = PRECO_DESCONTO
    df['Desconto (porcentagem)'] = DESCONTO 
    df['Código EAN'] = EAN
    df['Marca'] = MARCA
    df['Categoria'] = CATEGORIA
    print(df)

    return df

if __name__ == "__main__":
    df = main()


4560
https://www.farmaponte.com.br/dipirona-gts-10ml-ger/p
                                                   Nome Preço sem desconto  \
0                            Dipirona Gotas 10ml Germed               6.93   
1     Dipirona Monoidratada Biosintética - Aché 50mg...              16.78   
2     Citrato de Sildenafila Medley 50mg, caixa com ...              17.51   
3     Tadalafila Eurofarma 5mg, caixa com 30 comprim...             347.64   
4     Tadalafila Eurofarma 20mg, caixa com 4 comprim...             185.37   
...                                                 ...                ...   
4555           Filinar G 10mg ml xpe fr 120 ml+copo med              40.92   
4556  Coenzima Q10 Com Colageno Qualynutri, Com 30 C...              43.32   
4557              Selozok 50mg caixa com 60 comprimidos             154.94   
4558  Deocil Sl 10Mg, Caixa Com 20 Comprimidos Subli...              77.46   
4559        Noex Budesonida 64mcg Spray Nasal 120 doses              76.03   

    

In [2]:
lista_marcas = ["1Farma", "ADV Farma", "Abbott do Brasil", "Accord Farma", "Aché",
    "Addera", "Allergan", "Althaia", "Apsen", "Arese Pharma", "Aspen Pharma",
    "Astellas Farma", "Astrazeneca", "Baldacci", "Bausch + Lomb", "Bayer",
    "Besins", "Biolab", "Biolab Genéricos", "Biosintética - Aché", "Blanver S.A.",
    "Blau", "Boehringer Ingelheim", "Boiron", "Brasterápica", "Bravir",
    "Catarinense Pharma", "Cellera Farma", "Centrum", "Chiesi", "Cifarma",
    "Cimed", "Cosmed", "Daiichi-Sankyo", "Diffucap Chemobras", "EMS",
    "EMS Sigma Pharma", "Eli Lilly", "Eurofarma", "Exeltis", "Farmax",
    "Farmoquímica", "FiberMais", "GSK", "Galderma", "Geolab", "Germed Pharma",
    "Glenmark", "Granado Pharmácias", "Gross", "Grünenthal", "Gummy", "Hebron",
    "Herbarium", "Janssen-Cilag", "Kley Hertz Farmacêutica", "Laboratório Cristália",
    "Legrand", "Leo Pharma", "Libbs", "Marjan Farma", "Medley", "Medquímica",
    "Megalabs", "Merck", "Merck Sharp & Dohme", "Momenta Farma", "Multilab", "Mylan",
    "Myralis", "Natulab", "Neo Química", "Nova Química", "Novartis", "Novo Nordisk",
    "Organon", "P&G", "PF Consumer Healthcare", "Pfizer", "Pharlab", "Polinize Florais",
    "Prati-Donaduzzi", "Ranbaxy", "Reckitt Benckiser", "Roche", "Sandoz", "Sanofi",
    "Servier", "Supera", "Takeda", "Teuto", "Theraskin", "Top Beauty", "Torrent",
    "União Química", "Vick", "Vitamedic", "Wyeth/Pfizer", "Zambon", "Zodiac"]

In [3]:
def preencher_marca(df, lista_marcas):
    for i, nome in enumerate(df['Nome']):
        if pd.isna(df.at[i, 'Marca']):
            for marca in lista_marcas:
                if any(word.lower() in nome.lower() for word in marca.split()):
                    df.at[i, 'Marca'] = marca
                    break

In [4]:
dataframe = df.copy()

In [5]:
preencher_marca(dataframe, lista_marcas)

In [6]:
dataframe.head(50)

,Nome,Preço sem desconto,Preço com desconto,Desconto (porcentagem),Código EAN,Marca,Categoria
0,Dipirona Gotas 10ml Germed,6.93,2.08,70.0,7896004719115,Germed Pharma,Emagrecedores
1,Dipirona Monoidratada Biosintética - Aché 50mg...,16.78,16.78,0.0,7896181911036,Biosintética - Aché,Analgésicos
2,"Citrato de Sildenafila Medley 50mg, caixa com ...",17.51,14.01,20.0,7896422525190,Medley,Disfunção Erétil
3,"Tadalafila Eurofarma 5mg, caixa com 30 comprim...",347.64,27.81,92.0,7891317127725,Eurofarma,Disfunção Erétil
4,"Tadalafila Eurofarma 20mg, caixa com 4 comprim...",185.37,14.83,92.0,7891317127800,Eurofarma,Disfunção Erétil
5,"Apevitin BC 4mg, caixa com 1 frasco com 240mL ...",50.77,20.82,59.0,7896004700502,EMS,Similar
6,"Citrato de Sildenafila Eurofarma 50mg, caixa c...",30.62,13.78,55.0,7891317477592,Eurofarma,Disfunção Erétil
7,Materna Nestle 30 Capsula,89.9,89.9,0.0,7891045031257,Wyeth/Pfizer,Vitaminas
8,Odorizante De Ambiente Bendita Canfora 8 Tabletes,15.72,14.15,10.0,7896186002548,Bravir,Relaxante Muscular
9,Leite de Magnésia Eno frasco com 350mL de susp...,26.03,23.43,10.0,7896015591946,GSK,Digestivo


In [7]:
valores_nan = dataframe['Marca'].isna().sum()

print("Número de valores NaN na coluna 'Nome':", valores_nan)

Número de valores NaN na coluna 'Nome': 310


In [8]:
dataframe.columns

Index(['Nome', 'Preço sem desconto', 'Preço com desconto',
       'Desconto (porcentagem)', 'Código EAN', 'Marca', 'Categoria'],
      dtype='object')

In [9]:
dataframe.head()

,Nome,Preço sem desconto,Preço com desconto,Desconto (porcentagem),Código EAN,Marca,Categoria
0,Dipirona Gotas 10ml Germed,6.93,2.08,70.0,7896004719115,Germed Pharma,Emagrecedores
1,Dipirona Monoidratada Biosintética - Aché 50mg...,16.78,16.78,0.0,7896181911036,Biosintética - Aché,Analgésicos
2,"Citrato de Sildenafila Medley 50mg, caixa com ...",17.51,14.01,20.0,7896422525190,Medley,Disfunção Erétil
3,"Tadalafila Eurofarma 5mg, caixa com 30 comprim...",347.64,27.81,92.0,7891317127725,Eurofarma,Disfunção Erétil
4,"Tadalafila Eurofarma 20mg, caixa com 4 comprim...",185.37,14.83,92.0,7891317127800,Eurofarma,Disfunção Erétil


In [17]:
dataframe.to_excel('Drogaria_FarmaPonte.xlsx', index = False)